In [1]:
type Position = {
    line: int
    col: int
    height: int
}
type Location =
| Start of Position
| End of Position
| Step of Position
with
    override this.ToString() =
        match this with
        | Start s -> $"S{s}"
        | End e -> $"E{e}"
        | Step it -> $".{it}"

module Height =
    let value line col (character:char) =
        let toInt c = (c |> int) - 96
        let aPos = { line = line; col = col; height = -1 }
        match character with
        | 'S' -> Location.Start { aPos with height = 'a' |> toInt }
        | 'E' -> Location.End { aPos with height = 'z' |> toInt }
        | _ -> Location.Step { aPos with height = character |> toInt }

type Path = {
    up : Path option
    down : Path option
    right : Path option
    left : Path option
    location: Location
}

module Path =
    let mapIt values =
        values
        |> Seq.mapi (fun idx line -> line |> Seq.mapi (fun idy value -> value |> Height.value idx idy ))

    let start map =
        map
        |> Seq.concat
        |> Seq.filter (fun it -> match it with | Location.Start _ -> true | _ -> false)
        |> Seq.tryHead
    
    type Direction = UP | DOWN | LEFT | RIGHT
    let find path map =
        let follow direction from map =
            match direction with
            | UP -> None
            | DOWN -> None
            | LEFT -> None
            | RIGHT -> None

        let rec finder location path map =
            match location with
            | Some (Location.Start s) ->
                let up = map |> follow UP location
                None
            | Some (Location.End e) ->
                None
            | Some (Location.Step s) ->
                None
            | None ->
                None
        let start = map |> start

        map |> finder start None


let ResolutionFolder = __SOURCE_DIRECTORY__
let lines = File.ReadLines( ResolutionFolder + "/testcase12.txt")

let map =
    lines
    |> Path.mapIt
    |> Path.start

map

Value
Start Item: Position line: 0 col: 0 height: 1
